In [92]:
import pandas as pd
from pathlib import Path

In [93]:
# Loading data
file_path = Path("us_equities_news_dataset.csv")
df_news = pd.read_csv(file_path)
df_news.head()
print(df_news.shape)

(221513, 9)


In [94]:
df_NIO = df_news[df_news['ticker'] == "TSLA"]

In [95]:
print(df_NIO.shape)
df_NIO.head()

(4283, 9)


,id,ticker,title,category,content,release_date,provider,url,article_id
7191,228706,TSLA,Tesla TSLA Expected To Beat Earnings Estimat...,opinion,The market expects Tesla NASDAQ TSLA to deli...,2020-01-22,Zacks Investment Research,https://www.investing.com/analysis/tesla-tsla-...,200500661
7192,228707,TSLA,What s The Right Valuation Multiple For Tesla,opinion,In May of 2018 Tesla NASDAQ TSLA CEO Elon ...,2020-01-22,Zacks Investment Research,https://www.investing.com/analysis/what39s-the...,200500769
7193,228708,TSLA,Tesla NIO And The Electric Boogie,opinion,It s been a rough couple of months with the im...,2020-01-22,Zacks Investment Research,https://www.investing.com/analysis/tesla-nio-a...,200500783
7194,228709,TSLA,Toyota TM Recalls 3 4M Vehicles Worldwide Ov...,opinion,Toyota Motor Corporation NYSE TM will reca...,2020-01-23,Zacks Investment Research,https://www.investing.com/analysis/toyota-tm-r...,200500917
7199,228714,TSLA,Tesla TSLA Does This Rally Still Have Legs,opinion,Tesla NASDAQ TSLA bulls have had a field d...,2020-01-20,Zacks Investment Research,https://www.investing.com/analysis/tesla-tsla-...,200500132


In [96]:
df_NIO = df_NIO.drop(columns=["category","provider","url","article_id","title","id"])
df_NIO.head()


,ticker,content,release_date
7191,TSLA,The market expects Tesla NASDAQ TSLA to deli...,2020-01-22
7192,TSLA,In May of 2018 Tesla NASDAQ TSLA CEO Elon ...,2020-01-22
7193,TSLA,It s been a rough couple of months with the im...,2020-01-22
7194,TSLA,Toyota Motor Corporation NYSE TM will reca...,2020-01-23
7199,TSLA,Tesla NASDAQ TSLA bulls have had a field d...,2020-01-20


In [97]:
df_NIO = df_NIO.sort_values(by="release_date")

In [98]:
df_NIO.to_csv("TSLA_news.csv", index=False)



In [99]:
file_path = Path("TSLA_news.csv")
TSLA_news = pd.read_csv(file_path)
TSLA_news.head()

,ticker,content,release_date
0,TSLA,In today s bubble 2 0 world billions of VC do...,2012-05-23
1,TSLA,EquitiesAsian markets gained on Wednesday ahe...,2012-06-21
2,TSLA,Entrepreneurs that have the capacity to change...,2012-08-05
3,TSLA,Google Inc GOOG got caught naked yesterday ...,2012-10-19
4,TSLA,Recent Performance of the Stocks Mentioned Bel...,2012-11-19


In [100]:
print(TSLA_news.shape)

(4283, 3)


In [101]:
# Checking for null

In [102]:
# Determining number of nulls
TSLA_news.isnull().sum()

ticker          0
content         0
release_date    0
dtype: int64

In [103]:
TSLA_news_date = TSLA_news.drop(columns=["ticker","content"])
TSLA_news_date.head()



,release_date
0,2012-05-23
1,2012-06-21
2,2012-08-05
3,2012-10-19
4,2012-11-19


In [104]:
TSLA_news.dtypes


ticker          object
content         object
release_date    object
dtype: object

In [105]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
# Create the Bitcoin sentiment scores DataFrame
stock_sentiments = []

for article in TSLA_news['content']:
    try:
        text = article
        #date = article["release_date"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        stock_sentiments.append({
            "text": text,
            #"date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
stock_df = pd.DataFrame(stock_sentiments)

# Reorder DataFrame columns
cols = ["compound", "negative", "neutral","positive","text"]
stock_df = stock_df[cols]

stock_df.head()

,compound,negative,neutral,positive,text
0,0.9972,0.067,0.737,0.197,In today s bubble 2 0 world billions of VC do...
1,0.6808,0.067,0.836,0.097,EquitiesAsian markets gained on Wednesday ahe...
2,0.9971,0.034,0.822,0.144,Entrepreneurs that have the capacity to change...
3,0.9898,0.069,0.781,0.150,Google Inc GOOG got caught naked yesterday ...
4,0.9893,0.006,0.917,0.077,Recent Performance of the Stocks Mentioned Bel...


In [106]:
final_news_df = pd.concat([stock_df,TSLA_news_date],axis="columns", join="inner")
final_news_df.head()

,compound,negative,neutral,positive,text,release_date
0,0.9972,0.067,0.737,0.197,In today s bubble 2 0 world billions of VC do...,2012-05-23
1,0.6808,0.067,0.836,0.097,EquitiesAsian markets gained on Wednesday ahe...,2012-06-21
2,0.9971,0.034,0.822,0.144,Entrepreneurs that have the capacity to change...,2012-08-05
3,0.9898,0.069,0.781,0.150,Google Inc GOOG got caught naked yesterday ...,2012-10-19
4,0.9893,0.006,0.917,0.077,Recent Performance of the Stocks Mentioned Bel...,2012-11-19


In [107]:
final_news_df = final_news_df[["release_date","compound","negative","neutral","positive","text"]]
final_news_df.head()

,release_date,compound,negative,neutral,positive,text
0,2012-05-23,0.9972,0.067,0.737,0.197,In today s bubble 2 0 world billions of VC do...
1,2012-06-21,0.6808,0.067,0.836,0.097,EquitiesAsian markets gained on Wednesday ahe...
2,2012-08-05,0.9971,0.034,0.822,0.144,Entrepreneurs that have the capacity to change...
3,2012-10-19,0.9898,0.069,0.781,0.150,Google Inc GOOG got caught naked yesterday ...
4,2012-11-19,0.9893,0.006,0.917,0.077,Recent Performance of the Stocks Mentioned Bel...


In [108]:
columns = ["date","compound","negative","neutral","positive","text"]
final_news_df.columns = columns
final_news_df.head(10)

,date,compound,negative,neutral,positive,text
0,2012-05-23,0.9972,0.067,0.737,0.197,In today s bubble 2 0 world billions of VC do...
1,2012-06-21,0.6808,0.067,0.836,0.097,EquitiesAsian markets gained on Wednesday ahe...
2,2012-08-05,0.9971,0.034,0.822,0.144,Entrepreneurs that have the capacity to change...
3,2012-10-19,0.9898,0.069,0.781,0.150,Google Inc GOOG got caught naked yesterday ...
4,2012-11-19,0.9893,0.006,0.917,0.077,Recent Performance of the Stocks Mentioned Bel...
5,2012-11-28,0.2023,0.059,0.879,0.062,TSLA is trading 34 21 up 6 4 with IV30 dow...
6,2012-12-13,0.9995,0.002,0.862,0.136,Screen Criteria All of these stocks have see...
7,2013-02-08,0.8908,0.000,0.861,0.139,Within days I m picking up a Model S long si...
8,2013-02-11,-0.9092,0.093,0.826,0.082,Futures Rise Eyeing European PoliticsU S equ...
9,2013-02-17,0.9663,0.041,0.885,0.074,The third full week of February will be one of...


In [109]:
final_news_df.to_csv("final_news_df.csv", index=False)

In [110]:
file_path = Path("final_news_df.csv")
final_news_df_2 = pd.read_csv(file_path)
final_news_df_2.head(10)

,date,compound,negative,neutral,positive,text
0,2012-05-23,0.9972,0.067,0.737,0.197,In today s bubble 2 0 world billions of VC do...
1,2012-06-21,0.6808,0.067,0.836,0.097,EquitiesAsian markets gained on Wednesday ahe...
2,2012-08-05,0.9971,0.034,0.822,0.144,Entrepreneurs that have the capacity to change...
3,2012-10-19,0.9898,0.069,0.781,0.150,Google Inc GOOG got caught naked yesterday ...
4,2012-11-19,0.9893,0.006,0.917,0.077,Recent Performance of the Stocks Mentioned Bel...
5,2012-11-28,0.2023,0.059,0.879,0.062,TSLA is trading 34 21 up 6 4 with IV30 dow...
6,2012-12-13,0.9995,0.002,0.862,0.136,Screen Criteria All of these stocks have see...
7,2013-02-08,0.8908,0.000,0.861,0.139,Within days I m picking up a Model S long si...
8,2013-02-11,-0.9092,0.093,0.826,0.082,Futures Rise Eyeing European PoliticsU S equ...
9,2013-02-17,0.9663,0.041,0.885,0.074,The third full week of February will be one of...


In [111]:
# Import libraries and dependencies
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
# Import libraries and dependencies
from pathlib import Path
import plotly.express as px
import panel as pn

pn.extension()

import hvplot
import hvplot.pandas

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

%matplotlib inline

In [112]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [117]:
# Set the ticker
ticker = ["TSLA"]

# Set timeframe to '1D'
timeframe = "1D"

# Set start and end datetimes of 3 years from Today
start_date = pd.Timestamp("2010-09-04", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-12-30", tz="America/New_York").isoformat()

# Get 3 years worth of historical data for TSLA
ticker_data = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date
).df

# Drop the time component of the date

ticker_data.head(10)

TSLA                                 
                            open     high    low   close   volume
2010-09-07 00:00:00-04:00  20.61  21.0000  20.50  20.550   241115
2010-09-08 00:00:00-04:00  20.66  20.9500  20.60  20.858   286792
2010-09-09 00:00:00-04:00  21.00  21.0500  20.69  20.710   372192
2010-09-10 00:00:00-04:00  20.75  20.9300  19.76  20.180   385721
2010-09-13 00:00:00-04:00  20.89  20.8995  20.50  20.745   352237
2010-09-14 00:00:00-04:00  20.54  21.6000  20.53  21.110   644709
2010-09-15 00:00:00-04:00  20.98  22.0000  20.79  21.980   672894
2010-09-16 00:00:00-04:00  22.15  23.1600  20.84  20.960  2472897
2010-09-17 00:00:00-04:00  21.02  21.3200  19.80  20.190  1080715
2010-09-20 00:00:00-04:00  20.65  21.3500  20.16  21.060   940475

In [145]:
# Create and empty DataFrame for closing prices
df_closing_prices = pd.DataFrame()

# Fetch the closing prices of KO and TSLA
df_closing_prices["TSLA"] = ticker_data["TSLA"]["close"]

# Drop the time component of the date
df_closing_prices.index = df_closing_prices.index.date

# Compute daily returns
df_daily_returns = df_closing_prices.pct_change().dropna()

# Display sample data
df_daily_returns.head(10)

,TSLA
2010-09-08,0.014988
2010-09-09,-0.007096
2010-09-10,-0.025592
2010-09-13,0.027998
2010-09-14,0.017595
2010-09-15,0.041213
2010-09-16,-0.046406
2010-09-17,-0.036737
2010-09-20,0.043091
2010-09-21,-0.013295


In [146]:
#reset index to make date a column
df_daily_returns = df_daily_returns.reset_index().rename(columns = {"index":"date"})

df_daily_returns.head()

,date,TSLA
0,2010-09-08,0.014988
1,2010-09-09,-0.007096
2,2010-09-10,-0.025592
3,2010-09-13,0.027998
4,2010-09-14,0.017595


In [147]:
df_daily_returns.dtypes

#final_news_df_2 = pd.concat([NIO,final_news_df],axis="columns", join="inner")
#final_news_df_2.head()

date     object
TSLA    float64
dtype: object

,date,TSLA
0,2010-09-08,0
1,2010-09-09,0
2,2010-09-10,0
3,2010-09-13,0
4,2010-09-14,0
...,...,...
2592,2020-12-23,0
2593,2020-12-24,0
2594,2020-12-28,0
2595,2020-12-29,0


In [149]:
#adding close price to stock_df and matching close dates with article dates
final_news_df_2["Prices"] = ""

indx=0
for i in range (0,len(final_news_df_2)):
    for j in range (0,len(df_daily_returns)):
        get_news_date=final_news_df_2.date.iloc[i]
        get_stock_date=df_daily_returns.date.iloc[j]
        if(str(get_stock_date)==str(get_news_date)):
            #print(get_stock_date," ",get_tweet_date)
            final_news_df_2.at[i, 'Prices'] = (df_daily_returns.TSLA[j])
            break

In [150]:
final_news_df_2.head(10)

,date,compound,negative,neutral,positive,text,Prices
0,2012-05-23,0.9972,0.067,0.737,0.197,In today s bubble 2 0 world billions of VC do...,0.00746996
1,2012-06-21,0.6808,0.067,0.836,0.097,EquitiesAsian markets gained on Wednesday ahe...,-0.0482391
2,2012-08-05,0.9971,0.034,0.822,0.144,Entrepreneurs that have the capacity to change...,
3,2012-10-19,0.9898,0.069,0.781,0.150,Google Inc GOOG got caught naked yesterday ...,-0.0107343
4,2012-11-19,0.9893,0.006,0.917,0.077,Recent Performance of the Stocks Mentioned Bel...,0.0313873
5,2012-11-28,0.2023,0.059,0.879,0.062,TSLA is trading 34 21 up 6 4 with IV30 dow...,0.0335925
6,2012-12-13,0.9995,0.002,0.862,0.136,Screen Criteria All of these stocks have see...,-0.0479433
7,2013-02-08,0.8908,0.000,0.861,0.139,Within days I m picking up a Model S long si...,-0.00582573
8,2013-02-11,-0.9092,0.093,0.826,0.082,Futures Rise Eyeing European PoliticsU S equ...,-0.0216561
9,2013-02-17,0.9663,0.041,0.885,0.074,The third full week of February will be one of...,


In [151]:

final_news_df_2.dtypes
#final_news_df_2['Prices'] = final_news_df_2['Prices'].astype('int')
#final_news_df_2

date         object
compound    float64
negative    float64
neutral     float64
positive    float64
text         object
Prices       object
dtype: object

In [152]:
final_news_df_2.to_csv("tsla_cleansed.csv", index=False)